#### as the other notebook shows, we had to <i>manually</i> copy/past that entire api response into our code. who even does that? that's what computers are for! let's automate this.

In [4]:
import requests
import json
import os
import pandas as pd

# see all the columns in the jupyter preview window
pd.set_option("display.max_columns", None)

# make all the columns as wide as we want them
pd.set_option("display.max_colwidth", None)

# get our environment variable "cat_apikey"
cat_apikey = os.environ.get("cat_apikey")

# our target url
url = "https://api.thecatapi.com/v1/images/search?size=med&mime_types=jpg&format=json&has_breeds=true&order=RANDOM&page=0&limit=10"

# hold the empty dictionary of data we"re sending with the api request
payload={}

# header information for the api
# (none in this get request, but if ther was some, this would promise the api it would be in json format)
headers = {
  "Content-Type": "application/json",

# how the catapi likes to digest my api key called "cat_apikey"    
    "x-api-key": cat_apikey
}

# the meat of the api request
response = requests.request("GET", url, headers=headers, data=payload)

# turn the incoming string ("response") into a python object for pandas ("call_response")
call_response = response.json()

# turn said python object into its own pandas dataframe
api_df1 = pd.json_normalize(call_response, record_path=["breeds"], meta=["id", "url"], meta_prefix="image_")

# show me the first five rows of data
api_df1.head()

,id,name,cfa_url,vetstreet_url,vcahospitals_url,temperament,origin,country_codes,country_code,description,life_span,indoor,lap,alt_names,adaptability,affection_level,child_friendly,dog_friendly,energy_level,grooming,health_issues,intelligence,shedding_level,social_needs,stranger_friendly,vocalisation,experimental,hairless,natural,rare,rex,suppressed_tail,short_legs,wikipedia_url,hypoallergenic,reference_image_id,weight.imperial,weight.metric,cat_friendly,bidability,image_id,image_url
0,abob,American Bobtail,http://cfa.org/Breeds/BreedsAB/AmericanBobtail.aspx,http://www.vetstreet.com/cats/american-bobtail,https://vcahospitals.com/know-your-pet/cat-breeds/american-bobtail,"Intelligent, Interactive, Lively, Playful, Sensitive",United States,US,US,American Bobtails are loving and incredibly intelligent cats possessing a distinctive wild appearance. They are extremely interactive cats that bond with their human family with great devotion.,11 - 15,0,1.0,,5,5,4,5,3,1,1,5,3,3,3,3,0,0,0,0,0,1,0,https://en.wikipedia.org/wiki/American_Bobtail,0,hBXicehMA,7 - 16,3 - 7,NaN,NaN,gVrhv_yAY,https://cdn2.thecatapi.com/images/gVrhv_yAY.jpg
1,beng,Bengal,http://cfa.org/Breeds/BreedsAB/Bengal.aspx,http://www.vetstreet.com/cats/bengal,https://vcahospitals.com/know-your-pet/cat-breeds/bengal,"Alert, Agile, Energetic, Demanding, Intelligent",United States,US,US,"Bengals are a lot of fun to live with, but they're definitely not the cat for everyone, or for first-time cat owners. Extremely intelligent, curious and active, they demand a lot of interaction and woe betide the owner who doesn't provide it.",12 - 15,0,0.0,NaN,5,5,4,5,5,1,3,5,3,5,3,5,0,0,0,0,0,0,0,https://en.wikipedia.org/wiki/Bengal_(cat),1,O3btzLlsO,6 - 12,3 - 7,4.0,3.0,IFXsxmXLm,https://cdn2.thecatapi.com/images/IFXsxmXLm.jpg
2,beng,Bengal,http://cfa.org/Breeds/BreedsAB/Bengal.aspx,http://www.vetstreet.com/cats/bengal,https://vcahospitals.com/know-your-pet/cat-breeds/bengal,"Alert, Agile, Energetic, Demanding, Intelligent",United States,US,US,"Bengals are a lot of fun to live with, but they're definitely not the cat for everyone, or for first-time cat owners. Extremely intelligent, curious and active, they demand a lot of interaction and woe betide the owner who doesn't provide it.",12 - 15,0,0.0,NaN,5,5,4,5,5,1,3,5,3,5,3,5,0,0,0,0,0,0,0,https://en.wikipedia.org/wiki/Bengal_(cat),1,O3btzLlsO,6 - 12,3 - 7,4.0,3.0,UhqCZ7tC4,https://cdn2.thecatapi.com/images/UhqCZ7tC4.jpg
3,bsho,British Shorthair,http://cfa.org/Breeds/BreedsAB/BritishShorthair.aspx,http://www.vetstreet.com/cats/british-shorthair,https://vcahospitals.com/know-your-pet/cat-breeds/british-shorthair,"Affectionate, Easy Going, Gentle, Loyal, Patient, calm",United Kingdom,GB,GB,"The British Shorthair is a very pleasant cat to have as a companion, ans is easy going and placid. The British is a fiercely loyal, loving cat and will attach herself to every one of her family members. While loving to play, she doesn't need hourly attention. If she is in the mood to play, she will find someone and bring a toy to that person. The British also plays well by herself, and thus is a good companion for single people.",12 - 17,0,1.0,"Highlander, Highland Straight, Britannica",5,4,4,5,2,2,2,3,4,3,2,1,0,0,1,0,0,0,0,https://en.wikipedia.org/wiki/British_Shorthair,0,s4wQfYoEk,12 - 20,5 - 9,NaN,NaN,HYTJQgs2M,https://cdn2.thecatapi.com/images/HYTJQgs2M.jpg
4,char,Chartreux,http://cfa.org/Breeds/BreedsCJ/Chartreux.aspx,http://www.vetstreet.com/cats/chartreux,https://vcahospitals.com/know-your-pet/cat-breeds/chartreux,"Affectionate, Loyal, Intelligent, Social, Lively, Playful",France,FR,FR,"The Chartreux is generally silent but communicative. Short play sessions, mixed with naps and meals are their perfect day. Whilst appreciating any attention you give them, they are not demanding, content instead to follow you around devotedly, sleep on your bed and snuggle with you if you’re not feeling well.",12 - 15,0,1.0,,5,5,4,5,2,1,2,4,3,5,5,1,0,0,0,0,1,0,0,http

In [5]:
# check column names
print(api_df1.columns)

Index(['id', 'name', 'cfa_url', 'vetstreet_url', 'vcahospitals_url',
       'temperament', 'origin', 'country_codes', 'country_code', 'description',
       'life_span', 'indoor', 'lap', 'alt_names', 'adaptability',
       'affection_level', 'child_friendly', 'dog_friendly', 'energy_level',
       'grooming', 'health_issues', 'intelligence', 'shedding_level',
       'social_needs', 'stranger_friendly', 'vocalisation', 'experimental',
       'hairless', 'natural', 'rare', 'rex', 'suppressed_tail', 'short_legs',
       'wikipedia_url', 'hypoallergenic', 'reference_image_id',
       'weight.imperial', 'weight.metric', 'cat_friendly', 'bidability',
       'image_id', 'image_url'],
      dtype='object')


In [6]:
import pandas as pd
import requests
import json 
import os
from dotenv import load_dotenv

def snoogums():
    """
    function that takes the results from thecatapi call and puts them into a format that"s easier to read.
    the purpose for this is so that a prompt engineer can call this api and easily make out the information
    it is providing so they can write quicker, more in-depth prompts.
    """

    load_dotenv()
    
    cat_apikey = os.environ.get("cat_apikey")
    
    url = "https://api.thecatapi.com/v1/images/search?size=med&mime_types=jpg&format=json&has_breeds=true&order=RANDOM&page=0&limit=10"
    payload = {}
    headers = {
        "Content-Type": "application/json",
        "x-api-key": cat_apikey  # Make sure to include YOUR actual API key
    }
    response = requests.get(url, headers=headers, data=payload)

    # served as a checkpoint to see if the call was working
    # print(response.text)

    try:
        call_response = response.json()
    except json.JSONDecodeError as e:
        print(f"There was an error in deconding JSON: {e}")
        print(f"Response text: {response.text}")

        # exit exception block if the json decoding fails
        
        return None 

    api_df = pd.DataFrame(call_response)
    
    # good place to check how things are working
    # print(api_df)

    # Extract the 'breeds' column into a new DataFrame, applying pd.Series to turn each dictionary in the list into its own series
    breeds_df = api_df["breeds"]

    # google's gemini helped in the following code:
    # it mentioned that there may be times when multiple breeds are in a single entry
    # and what to do if that happens. didn't catch that myself, and credit where credit's due!
    if any(breeds_df.apply(len) > 1):
        breeds_df = breeds_df.explode(column=0) # Explode specific column

    breeds_df = pd.json_normalize(breeds_df)

    # now that 'breeds' has been cleaned up, we can join (concatenate)
    # it with the api_df, from which 'breeds' was removed
    api_df = pd.concat([api_df.drop('breeds', axis=1), breeds_df], axis=1)


    print("Original DataFrame with Flattened Breeds:")
    print(api_df.head().to_string(index=False, line_width=1000))

    return api_df # Returns the modified dataframe

In [7]:
snoogums()

Original DataFrame with Flattened Breeds:
       id                                             url  width  height  \
HVP1wfZF0 https://cdn2.thecatapi.com/images/HVP1wfZF0.jpg   1600    1067   
-y_6SBPrw https://cdn2.thecatapi.com/images/-y_6SBPrw.jpg   1280     959   
-tm9-znzl https://cdn2.thecatapi.com/images/-tm9-znzl.jpg   1125     750   
KUcr7eFlM https://cdn2.thecatapi.com/images/KUcr7eFlM.jpg   2405    1549   
Ttk_tdV4g https://cdn2.thecatapi.com/images/Ttk_tdV4g.jpg   1152     768   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

,id,url,width,height,0
0,HVP1wfZF0,https://cdn2.thecatapi.com/images/HVP1wfZF0.jpg,1600,1067,"{'id': 'acur', 'name': 'American Curl', 'cfa_u..."
1,-y_6SBPrw,https://cdn2.thecatapi.com/images/-y_6SBPrw.jpg,1280,959,"{'id': 'cymr', 'name': 'Cymric', 'vetstreet_ur..."
2,-tm9-znzl,https://cdn2.thecatapi.com/images/-tm9-znzl.jpg,1125,750,"{'id': 'jbob', 'name': 'Japanese Bobtail', 'cf..."
3,KUcr7eFlM,https://cdn2.thecatapi.com/images/KUcr7eFlM.jpg,2405,1549,"{'id': 'norw', 'name': 'Norwegian Forest Cat',..."
4,Ttk_tdV4g,https://cdn2.thecatapi.com/images/Ttk_tdV4g.jpg,1152,768,"{'id': 'siam', 'name': 'Siamese', 'cfa_url': '..."
5,_DFVS4T4I,https://cdn2.thecatapi.com/images/_DFVS4T4I.jpg,1080,1224,"{'id': 'sibe', 'name': 'Siberian', 'cfa_url': ..."
6,wUxNIew6I,https://cdn2.thecatapi.com/images/wUxNIew6I.jpg,1280,1920,"{'id': 'soma', 'name': 'Somali', 'cfa_url': 'h..."
7,oR3LMBqEZ,https://cdn2.thecatapi.com/images/oR3LMBqEZ.jpg,750,937,"{'id': 'tang', 'name': 'Turkish Angora', 'cfa_..."
8,PlvdElgfk,https://cdn2.thecatapi.com/images/PlvdElgfk.jpg,1080,1080,"{'id': 'ycho', 'name': 'York Chocolate', 'temp..."
9,QAWwhRFCG,https://cdn2.thecatapi.com/images/QAWwhRFCG.jpg,900,675,"{'id': 'sibe', 'name': 'Siberian', 'cfa_url': ..."
